#Motif Grafting
<font color='grey' size='1.5'> Created by Parisa Hosseinzadeh for *Protein Engineering and Design*, Winter 2022


In this exercise, you will be using a very powerful method for generating *de novo* binders starting from native motifs. This method is called **motif grafting**. You can read about how it works [here](https://new.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/movers_pages/MotifGraftMover).


### Preparing Pyrosetta on your computer



1.   Create a folder in your google drive and name it **PyRosetta**
2.   Download `pyrosetta-2020.50.post0.dev0+970.commits.3700df14560-cp37-cp37m-linux_x86_64.whl` from your course files and put it in the PyRosetta folder.



In [1]:
#@title Mounting Google Drive
#@markdown Please execute this cell by pressing the _Play_ button 
#@markdown on the left. 


google_drive_mount_point = '/content/google_drive'

import os, sys, time

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount(google_drive_mount_point)

if not os.getenv("DEBUG"):
    google_drive = google_drive_mount_point + '/My Drive' 

Mounted at /content/google_drive


In [2]:
#@title Installing PyRosetta
#@markdown Download pyrosetta. 
#@markdown This cell will take a few min to run.

%%time
if not os.getenv("DEBUG"):
    # installing PyRosetta
    if sys.version_info.major != 3 or sys.version_info.minor != 7:
        print('Need Python-3.7 to run!')
        sys.exit(1)

    # upload PyRosetta Linux WHEEL package into your google drive and put it into /PyRosetta dir
    # or alternatively you can download PyRosetta directly from GrayLab web site (but this might take some time!)
    #!mkdir $notebook_path/PyRosetta
    #!cd $notebook_path/PyRosetta && wget --user USERNAME --password PASSWORD https://graylab.jhu.edu/download/PyRosetta4/archive/release/PyRosetta4.Release.python37.ubuntu.wheel/latest.html   

    pyrosetta_distr_path = google_drive + '/PyRosetta' 
    
    # finding path to wheel package, if multiple packages is found take first one
    # replace this with `wheel_path = pyrosetta_distr_path + /<wheel-file-name>.whl` if you want to use particular whl file
    wheel_path = pyrosetta_distr_path + '/' + [ f for f in os.listdir(pyrosetta_distr_path) if f.endswith('.whl')][0]
    
    print(f'Using PyRosetta wheel package: {wheel_path}')

    !pip3 install '{wheel_path}' 

Using PyRosetta wheel package: /content/google_drive/My Drive/PyRosetta/pyrosetta-2020.50.post0.dev0+970.commits.3700df14560-cp37-cp37m-linux_x86_64.whl
Processing ./google_drive/My Drive/PyRosetta/pyrosetta-2020.50.post0.dev0+970.commits.3700df14560-cp37-cp37m-linux_x86_64.whl
CPU times: user 294 ms, sys: 60.8 ms, total: 355 ms
Wall time: 38.2 s


In [3]:
#@title Importing necessary modules
#@markdown Run this cell to download 
#@markdown necessary modules to run the code.

#importing modules necessary for plotting
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#importing the initialization module
from pyrosetta import init
#importing pose and score function getters
from pyrosetta import pose_from_file
#importing necessary modules for reading XML
from pyrosetta.rosetta.protocols.rosetta_scripts import XmlObjects
import pyrosetta.rosetta.protocols.rosetta_scripts as rosetta_scripts

#required for visualization
!pip install -q py3Dmol
import py3Dmol

In [4]:
#@title Visualizing the motif at the interface 
#@markdown By running this cell, you will see the interface
#@markdown we're optimizing.

#@markdown Please make sure you added the file comp_ssm.pdb
#@markdown to your colab. You can do this by clicking on the
#@markdown folder icon on the left bar and drag-drop the file.

#@markdown Open the same pdb in pymol. Do they look the same?

p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
p.addModel(open("motif.pdb",'r').read(),'pdb')
p.addModel(open("context.pdb",'r').read(),'pdb')
p.setStyle({'cartoon': {'color':'spectrum'}})
p.zoomTo()
p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
#@title Loading molecules
#@markdown We will prepare for the run by loading pdbs.
#@markdown For grafting you will need 3 molecules:
#@markdown 1. **motif** which is the small binding segment, 
#@markdown 2. **context** which is the target of interest,
#@markdown 3. **scaffolds** which are the final binding molecules

#@markdown The motif and context are read from the file and
#@markdown and the scaffolds are loaded.

#initializing Rosetta. Necessary to start
#init(extra_options='-mute all')
init("-mute all")
#reading in the pdb file
p = pose_from_file('scaffold.pdb')

PyRosetta-4 2021 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2020.50.post.dev+970.commits.3700df145608444753aabbec9c4681ec9b21f74b 2021-02-24T13:24:53] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


In [6]:
#@title Running grafting
#@markdown By running this cell, you perform the grafting.
#@markdown The code embedded here is a **Rosetta XML** file.
#@markdown At the end of this run, you will have a new pdb called
#@markdown *grafted_scaffold.pdb* generated in your folder.

#@markdown This method often generates multiple
#@markdown possible outputs. Today, we will
#@markdown be looking at only one output.

#creating XML object
xml_string ='''
    <ROSETTASCRIPTS>
    <SCOREFXNS>
        <ScoreFunction name="default" weights="ref2015.wts"/>
    </SCOREFXNS>
    <RESIDUE_SELECTORS>
    </RESIDUE_SELECTORS>
    <TASKOPERATIONS>
    </TASKOPERATIONS>
    <SIMPLE_METRICS>
    </SIMPLE_METRICS>
    <FILTERS>
    </FILTERS>
    <MOVERS>
        <MotifGraft name="motif_grafting"
            context_structure="context.pdb"
            motif_structure="motif.pdb"
            RMSD_tolerance="1.5"
            clash_score_cutoff="5"
            clash_test_residue="ALA"
            full_motif_bb_alignment="1"
            revert_graft_to_native_sequence="1"
            combinatory_fragment_size_delta="1:1"
            allow_repeat_same_graft_output="0" />
    </MOVERS>
    <PROTOCOLS>
        <Add mover="motif_grafting"/>
    </PROTOCOLS>
    <OUTPUT />
    </ROSETTASCRIPTS>
'''

#running
rosetta_scripts.XmlObjects.create_from_string(
    xml_string).get_mover("motif_grafting").apply(p)

p.dump_pdb("grafted_scaffold.pdb")

True

In [7]:
#@title Visualizing the result
#@markdown check the grafted scaffold
#@markdown you can download it from your folder 
#@markdown for future visualization.

p = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
p.addModel(open("grafted_scaffold.pdb",'r').read(),'pdb')
p.setStyle({'cartoon': {'color':'spectrum'}})
p.zoomTo()
p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol